In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id:  26637


In [3]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [4]:
#get only the shotsData
strings = scripts[1].string

In [5]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22590771\\x22,\\x22minute\\x22\\x3A\\x2212\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.975\\x22,\\x22Y\\x22\\x3A\\x220.39\\x22,\\x22xG\\x22\\x3A\\x220.02508147992193699\\x22,\\x22player\\x22\\x3A\\x22Ilkay\\x20G\\x5Cu00fcndogan\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x22314\\x22,\\x22situation\\x22\\x3A\\x22FromCorner\\x22,\\x22season\\x22\\x3A\\x222024\\x22,\\x22shotType\\x22\\x3A\\x22Head\\x22,\\x22match_id\\x22\\x3A\\x2226637\\x22,\\x22h_team\\x22\\x3A\\x22Manchester\\x20City\\x22,\\x22a_team\\x22\\x3A\\x22Brentford\\x22,\\x22h_goals\\x22\\x3A\\x222\\x22,\\x22a_goals\\x22\\x3A\\x221\\x22,\\x22date\\x22\\x3A\\x222024\\x2D09\\x2D14\\x2014\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Kevin\\x20De\\x20Bruyne\\x22,\\x22lastAction\\x22\\x3A\\x22Cross\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22590772\\x22,\\x22minute\\x22\\x3A\\x2219\\x22,\\x22result\\x22

In [6]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [7]:
data

{'h': [{'id': '590771',
   'minute': '12',
   'result': 'MissedShots',
   'X': '0.975',
   'Y': '0.39',
   'xG': '0.02508147992193699',
   'player': 'Ilkay Gündogan',
   'h_a': 'h',
   'player_id': '314',
   'situation': 'FromCorner',
   'season': '2024',
   'shotType': 'Head',
   'match_id': '26637',
   'h_team': 'Manchester City',
   'a_team': 'Brentford',
   'h_goals': '2',
   'a_goals': '1',
   'date': '2024-09-14 14:00:00',
   'player_assisted': 'Kevin De Bruyne',
   'lastAction': 'Cross'},
  {'id': '590772',
   'minute': '19',
   'result': 'Goal',
   'X': '0.9159999847412109',
   'Y': '0.39299999237060546',
   'xG': '0.06925240904092789',
   'player': 'Erling Haaland',
   'h_a': 'h',
   'player_id': '8260',
   'situation': 'OpenPlay',
   'season': '2024',
   'shotType': 'RightFoot',
   'match_id': '26637',
   'h_team': 'Manchester City',
   'a_team': 'Brentford',
   'h_goals': '2',
   'a_goals': '1',
   'date': '2024-09-14 14:00:00',
   'player_assisted': None,
   'lastAction': '

In [8]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [9]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [10]:
df

,x,y,xG,result,team
0,0.975,0.39,0.02508147992193699,MissedShots,Manchester City
1,0.9159999847412109,0.39299999237060546,0.06925240904092789,Goal,Manchester City
2,0.730999984741211,0.43099998474121093,0.018106698989868164,MissedShots,Manchester City
3,0.8730000305175781,0.5729999923706055,0.20122335851192474,Goal,Manchester City
4,0.865,0.4129999923706055,0.033002015203237534,BlockedShot,Manchester City
5,0.8059999847412109,0.32799999237060545,0.029425349086523056,SavedShot,Manchester City
6,0.8959999847412109,0.5729999923706055,0.03643355891108513,BlockedShot,Manchester City
7,0.8290000152587891,0.43599998474121093,0.04615657031536102,MissedShots,Manchester City
8,0.8209999847412109,0.6659999847412109,0.04012800753116608,SavedShot,Manchester City
9,0.8830000305175781,0.4379999923706055,0.11676666885614395,SavedShot,Manchester City


In [11]:
# Initialize an empty list to hold all rows of data
all_data = []

# Define a function to extract all fields from both home and away dictionaries
def extract_all_data(data, team_type):
    for event in data:
        # Create a dictionary for each event and add the team type
        event_data = event.copy()  # Make a copy of the original dictionary
        event_data['team'] = event_data[team_type]  # Add team field based on h_team or a_team
        del event_data[team_type]  # Remove the original h_team/a_team field
        all_data.append(event_data)  # Append to the main list

# Extract data for home and away teams
extract_all_data(data['h'], 'h_team')
extract_all_data(data['a'], 'a_team')

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(all_data)


In [12]:
df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,...,shotType,match_id,a_team,h_goals,a_goals,date,player_assisted,lastAction,team,h_team
0,590771,12,MissedShots,0.975,0.39,0.02508147992193699,Ilkay Gündogan,h,314,FromCorner,...,Head,26637,Brentford,2,1,2024-09-14 14:00:00,Kevin De Bruyne,Cross,Manchester City,NaN
1,590772,19,Goal,0.9159999847412109,0.39299999237060546,0.06925240904092789,Erling Haaland,h,8260,OpenPlay,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,None,Dispossessed,Manchester City,NaN
2,590773,25,MissedShots,0.730999984741211,0.43099998474121093,0.018106698989868164,Kevin De Bruyne,h,447,OpenPlay,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Rico Lewis,Pass,Manchester City,NaN
3,590776,31,Goal,0.8730000305175781,0.5729999923706055,0.20122335851192474,Erling Haaland,h,8260,OpenPlay,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Ederson,Pass,Manchester City,NaN
4,590777,33,BlockedShot,0.865,0.4129999923706055,0.033002015203237534,Sávio,h,11735,FromCorner,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Kevin De Bruyne,Chipped,Manchester City,NaN
5,590778,36,SavedShot,0.8059999847412109,0.32799999237060545,0.029425349086523056,Sávio,h,11735,OpenPlay,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Ilkay Gündogan,Pass,Manchester City,NaN
6,590779,40,BlockedShot,0.8959999847412109,0.5729999923706055,0.03643355891108513,Jack Grealish,h,675,FromCorner,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,None,Aerial,Manchester City,NaN
7,590782,64,MissedShots,0.8290000152587891,0.43599998474121093,0.04615657031536102,Sávio,h,11735,FromCorner,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Ilkay Gündogan,Pass,Manchester City,NaN
8,590783,66,SavedShot,0.8209999847412109,0.6659999847412109,0.04012800753116608,Jack Grealish,h,675,OpenPlay,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Kevin De Bruyne,Pass,Manchester City,NaN
9,590784,68,SavedShot,0.8830000305175781,0.4379999923706055,0.11676666885614395,Erling Haaland,h,8260,OpenPlay,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Ilkay Gündogan,Pass,Manchester City,NaN


In [28]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

# Function to extract data from a single match
def extract_match_data(match_id, base_url='https://understat.com/match/'):
    url = base_url + str(match_id)
    
    # Make the request and parse the page
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    
    # Get the script containing the shot data
    strings = scripts[1].string

    # Extract the JSON part from the script
    ind_start = strings.index("('") + 2 
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')

    # Convert the extracted string into JSON
    data = json.loads(json_data)

    # Initialize an empty list to hold all rows of data for this match
    match_data = []

    # Define a function to extract all fields from both home and away dictionaries
    def extract_all_data(data, team_type):
        for event in data:
            # Create a dictionary for each event and add the team type
            event_data = event.copy()  # Make a copy of the original dictionary
            event_data['team'] = event_data[team_type]  # Add team field based on h_team or a_team
            del event_data[team_type]  # Remove the original h_team/a_team field
            match_data.append(event_data)  # Append to the main list

    # Extract data for home and away teams
    extract_all_data(data['h'], 'h_team')
    extract_all_data(data['a'], 'a_team')

    # Return the match data as a list of dictionaries
    return match_data

# Function to process multiple matches
def extract_multiple_matches(match_ids, base_url='https://understat.com/match/', delay=5):
    all_matches_data = []
    
    for match_id in match_ids:
        print(f"Processing match ID: {match_id}")
        
        # Extract data for the current match
        match_data = extract_match_data(match_id, base_url)
        all_matches_data.extend(match_data)
        
        # Add a delay between requests to reduce load on the website
        time.sleep(delay)
    
    # Convert the combined list of dictionaries into a DataFrame
    df = pd.DataFrame(all_matches_data)
    
    # Return the DataFrame
    return df

# Example usage
match_ids = ['26637', '26628', '26615', '26610']  # List of match IDs to process
df = extract_multiple_matches(match_ids)

Processing match ID: 26637
Processing match ID: 26628
Processing match ID: 26615
Processing match ID: 26610


In [29]:
df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,...,shotType,match_id,a_team,h_goals,a_goals,date,player_assisted,lastAction,team,h_team
0,590771,12,MissedShots,0.975,0.39,0.02508147992193699,Ilkay Gündogan,h,314,FromCorner,...,Head,26637,Brentford,2,1,2024-09-14 14:00:00,Kevin De Bruyne,Cross,Manchester City,NaN
1,590772,19,Goal,0.9159999847412109,0.39299999237060546,0.06925240904092789,Erling Haaland,h,8260,OpenPlay,...,RightFoot,26637,Brentford,2,1,2024-09-14 14:00:00,None,Dispossessed,Manchester City,NaN
2,590773,25,MissedShots,0.730999984741211,0.43099998474121093,0.018106698989868164,Kevin De Bruyne,h,447,OpenPlay,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Rico Lewis,Pass,Manchester City,NaN
3,590776,31,Goal,0.8730000305175781,0.5729999923706055,0.20122335851192474,Erling Haaland,h,8260,OpenPlay,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Ederson,Pass,Manchester City,NaN
4,590777,33,BlockedShot,0.865,0.4129999923706055,0.033002015203237534,Sávio,h,11735,FromCorner,...,LeftFoot,26637,Brentford,2,1,2024-09-14 14:00:00,Kevin De Bruyne,Chipped,Manchester City,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,585316,37,BlockedShot,0.874000015258789,0.6780000305175782,0.046097420156002045,Josko Gvardiol,a,9790,FromCorner,...,LeftFoot,26610,NaN,0,2,2024-08-18 15:30:00,Jéremy Doku,Pass,Manchester City,Chelsea
92,585318,39,SavedShot,0.895,0.6769999694824219,0.08055149018764496,Bernardo Silva,a,3635,OpenPlay,...,LeftFoot,26610,NaN,0,2,2024-08-18 15:30:00,Kevin De Bruyne,Pass,Manchester City,Chelsea
93,585321,51,SavedShot,0.865999984741211,0.4759999847412109,0.11672431230545044,Erling Haaland,a,8260,OpenPlay,...,LeftFoot,26610,NaN,0,2,2024-08-18 15:30:00,Bernardo Silva,Pass,Manchester City,Chelsea
94,585325,72,MissedShots,0.7809999847412109,0.47400001525878904,0.046891916543245316,Kevin De Bruyne,a,447,OpenPlay,...,RightFoot,26610,NaN,0,2,2024-08-18 15:30:00,Bernardo Silva,Pass,Manchester City,Chelsea


In [31]:
haaland_2024 = df[df['player'] == 'Erling Haaland']

In [32]:
haaland_2024.to_csv('../data/raw/erling_haaland_2024_4games_understat.csv')